In [1]:
using Revise
using Pkg

ENV["PYTHON"] = Sys.which("python")
ENV["PYCALL_JL_RUNTIME_PYTHON"] = Sys.which("python")
Pkg.build("PyCall")
using FileIO
using JLD2
include("../src/DistributionallyRobust.jl")
using .DistributionallyRobust

    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/51cab8e982c5b598eea9c8ceaced4b58d9dd37c9/build.log`


    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/9816a3826b0ebf49ab4926e2b18842ad8b5c8f04/build.log`


┌ Error: Unable to load the 'rospy' python package!
│ Has an environment setup script been run?
└ @ RobotOS /home/kh-ryu/.julia/packages/RobotOS/j0Tsl/src/RobotOS.jl:35


┌ Warning: You are using Matplotlib 3.3.4, which is no longer
│ officially supported by the Plots community. To ensure smooth Plots.jl
│ integration update your Matplotlib library to a version >= 3.4.0
│ 
│ If you have used Conda.jl to install PyPlot (default installation),
│ upgrade your matplotlib via Conda.jl and rebuild the PyPlot.
│ 
│ If you are not sure, here are the default instructions:
│ 
│ In Julia REPL:
│ ```
│ import Pkg;
│ Pkg.add("Conda")
│ import Conda
│ Conda.update()
│ Pkg.build("PyPlot")
│ ```
└ @ Plots /home/kh-ryu/.julia/packages/Plots/sxUvK/src/backends/deprecated/pyplot.jl:16


┌ Warning: You are using Matplotlib 3.3.4, which is no longer
│ officially supported by the Plots community. To ensure smooth Plots.jl
│ integration update your Matplotlib library to a version >= 3.4.0
│ 
│ If you have used Conda.jl to install PyPlot (default installation),
│ upgrade your matplotlib via Conda.jl and rebuild the PyPlot.
│ 
│ If you are not sure, here are the default instructions:
│ 
│ In Julia REPL:
│ ```
│ import Pkg;
│ Pkg.add("Conda")
│ import Conda
│ Conda.update()
│ Pkg.build("PyPlot")
│ ```
└ @ Plots /home/kh-ryu/.julia/packages/Plots/sxUvK/src/backends/deprecated/pyplot.jl:16


┌ Info: Number of Julia Thread(s): 1
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/DistributionallyRobust.jl:39
┌ Info: CUDA Device: NVIDIA GeForce RTX 3060
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/DistributionallyRobust.jl:40
┌ Info: Python executable used by PyCall: /home/kh-ryu/anaconda3/envs/temp_trajectron/bin/python
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/DistributionallyRobust.jl:41


In [2]:
include("$(@__DIR__)/../scripts/default_params/params_drc_data_trajectron.jl");

epsilon = 0.15;

# test_data_name = "hotel_test.pkl";                                                  # test data set name
# test_scene_id = 0;                                                                  # test data id
# start_time_idx = 401;                                                               # start time index in test data
# ego_pos_init_vec = [-1.5, -8.5] .+ [-1.393743, 2.978962];                           # initial ego position [x, y] [m]
# ego_pos_goal_vec = [3.5, 0.0]   .+ [-1.393743, 2.978962];                           # goal ego position [x, y] [m]
# target_speed = 1.0;                                                                 # target speed [m/s]
# sim_horizon = 10.0;       

include("$(@__DIR__)/../scripts/parameter_setup_drc.jl");

In [3]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
    controller_setup(scene_param,
                    predictor_param,
                    prediction_device=prediction_device,
                    cost_param=cost_param,
                    cnt_param=cnt_param,
                    dtc=dtc,
                    ego_pos_init_vec=ego_pos_init_vec,
                    ego_pos_goal_vec=ego_pos_goal_vec,
                    target_speed=target_speed,
                    sim_horizon=sim_horizon,
                    verbose=true);

Scene Mode: data
Prediction Mode: trajectron
Deterministic Prediction: false
Loaded evaluation data from /home/kh-ryu/DRC_Project/DRCC-MPC/Trajectron-plus-plus/experiments/processed/eth_test.pkl
Looking at the eth_test.pkl sequence, data_id 0, start_idx 905


Loaded Trajectron model from /home/kh-ryu/DRC_Project/DRCC-MPC/Trajectron-plus-plus/experiments/pedestrians/models/eth_attention_radius_3/model_registrar-100.pt


In [4]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, ego_pos_goal_vec,
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan);

┌ Warning: Time 0.00 [sec]: DRC computation took 15.574 [sec], which exceeds the maximum computation time allowed.
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/distributionally_robust_controller.jl:254


┌ Warning: Time 0.10 [sec]: DRC computation took 0.13 [sec], which exceeds the maximum computation time allowed.
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/distributionally_robust_controller.jl:254
┌ Warning: Time 0.10 [sec]: This evaluation iteration took 0.13 [sec], which exceeds dtc.
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/drc_evaluation.jl:204


┌ Warning: Time 0.40 [sec]: This evaluation iteration took 0.105 [sec], which exceeds dtc.
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/drc_evaluation.jl:204


┌ Warning: Time 3.60 [sec]: All samples violate CVaR constraints.
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/distributionally_robust_controller.jl:315
┌ Warning: Time 3.60 [sec]: This evaluation iteration took 0.113 [sec], which exceeds dtc.
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/drc_evaluation.jl:204
┌ Warning: Time 3.70 [sec]: All samples violate CVaR constraints.
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/distributionally_robust_controller.jl:315


┌ Warning: Time 3.80 [sec]: All samples violate CVaR constraints.
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/distributionally_robust_controller.jl:315


...

┌ Warning: Time 8.00 [sec]: This evaluation iteration took 0.102 [sec], which exceeds dtc.
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/drc_evaluation.jl:204


┌ Warning: Time 8.40 [sec]: This evaluation iteration took 0.102 [sec], which exceeds dtc.
└ @ Main.DistributionallyRobust /home/kh-ryu/DRC_Project/DRCC-MPC/src/drc_evaluation.jl:204


/home/kh-ryu/anaconda3/envs/temp_trajectron/lib/python3.6/site-packages/torch/distributions/distribution.py:46: UserWarning: <class 'model.components.gmm2d.GMM2D'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  'with `validate_args=False` to turn off validation.')
/home/kh-ryu/anaconda3/envs/temp_trajectron/lib/python3.6/site-packages/torch/distributions/distribution.py:46: UserWarning: <class 'model.components.gmm2d.GMM2D'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  'with `validate_args=False` to turn off validation.')
/home/kh-ryu/anaconda3/envs/temp_trajectron/lib/python3.6/site-packages/torch/distributions/distribution.py:46: UserWarning: <class 'model.components.gmm2d.GMM2D'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distrib

Average computation time: 0.0521284008026123
std of computation time: 0.03150739558147522
List of computation time: Any

[0.054913997650146484, 0.12982702255249023, 0.050015926361083984, 0.07581210136413574, 0.10468697547912598, 0.05466794967651367, 0.027200937271118164, 0.03525710105895996, 0.07886219024658203, 0.03617405891418457, 0.030245065689086914, 0.035848140716552734, 0.08164215087890625, 0.030414104461669922, 0.027589082717895508, 0.030488967895507812, 0.09415698051452637, 0.03541994094848633, 0.033477067947387695, 0.03320884704589844, 0.09758496284484863, 0.036199092864990234, 0.03463387489318848, 0.04599404335021973, 0.08854293823242188, 0.03351593017578125, 0.03447604179382324, 0.03951597213745117, 0.08027911186218262, 0.030380964279174805, 0.02679610252380371, 0.030750036239624023, 0.051614999771118164, 0.018130064010620117, 0.017763137817382812, 0.029371976852416992, 0.11337709426879883, 0.037548065185546875, 0.03803300857543945, 0.03084707260131836, 0.12134504318237305, 0.03375697135925293, 0.038789987564086914, 0.043663978576660156, 0.11421823501586914, 0.038559913635253906, 0.04228901863

In [5]:
# display_log(result.log)

In [6]:
result.total_cnt_cost

1.2616577489286056

In [7]:
result.total_pos_cost

125.15151123760054

In [8]:
result.total_col_cost

14.32878045200265

In [9]:
result.total_col

0

In [10]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

140.7419494385318

In [11]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

0.8505075239880383

In [12]:
save("9_data_trajectron.jld2", "result", result)